<h1 style="font-size:40px;"><center>Exercise III: Regression
    
</center></h1>

Now we are going to look at a regression problem. The data as described above (regr1) consists of 6 inputs (features) and one output (target) value. As for previous examples a new data set is generated each time you call the *regr1* function. To get exactly the same data set between different calls, use a fixed seed. New for this problem is that one can also control the amount of noise added to the target value. We are going to use a relatively small training dataset (~250) and a larger validation dataset (~1000) to get a more robust estimation of the generalization performance. For regression problems we also need new performance measures. The *stats_reg* function will give you two such measures:
* MSE = mean squared error (low error mean good performance)
* CorrCoeff = Pearson correlation coefficient for the scatter plot between predicted and true values.

The cell below can be used as an template for all questions regarding this regression problem.

# Data 
## regr1
There is also a synthetic regression problem, called *regr1*. It has 6 inputs (independent variables) and one output variable (dependent variable). It is generated according to the following formula:  

$\qquad d = 2x_1 + x_2x_3^2 + e^{x_4} + 5x_5x_6 + 3\sin(2\pi x_6) + \alpha\epsilon$  
    
where $\epsilon$ is added normally distributed noise and $\alpha$ is a parameter controlling the size of the added noise. Variables $x_1,...,x_4$ are normally distrubuted with zero mean and unit variance, whereas $x_5, x_6$ are uniformly distributed ($[0,1]$). The target value $d$ has a non-linear dependence on ***x***.


# Tasks

## Task 1
Use 250 data points for training and about 1000 for validation and **no** added noise. Train an MLP to predict the target output. If you increase the complexity of the model (e.g. number of hidden nodes) you should be able to reach a very small training error. You will also most likely see that the validation error decreases as you increase the complexity or at least no clear sign of overtraining. 

**Note:** As with previous examples you may need to tune the optimization parameters to make sure that you have "optimal" training. That is, increase or decrease the learningrate, possibly train longer times (increase *epochs*) and change the *batch_size* parameter.

**Question:** Even though the validation error is most likely still larger than the training error why do we not see any overtraining of the model? (Hint: What is it that typically causes overfitting?)

## Task 2
Use the same training and validation data sets as above, but add 0.4 units of noise (set the second parameter when calling the *regr1* function to 0.4 for both training and validation). Now train again, starting with a "small" model and increase the number of hidden nodes as you monitor the validation result for each model. Make a note of the validation error you obtained a this point!

**Question:** How many nodes do you have for opitimal validation performance, i.e. more hidden nodes results in overtraining?

## Task 3
Instead of using the number of hidden nodes to control the complexity it is often better to use a regularization term added to the error function. You are now going to control the complexity by adding a *L2* regularizer (see the "INPUT" dictionary in the cell). You should modify this value until you find the "near optimal" validation performance. Use 15 hidden nodes. 

**Question:** Give the L2 value that you found to give "optimal" validation performance and compare with the result from  question 7 (optimal performance).

## Task 4
Summarize your findings in a few sentences.


In [58]:


# Generate training and validation data
x_train, d_train = MLPData.regr1(250, 0.0) # 250 data points with no noise
x_val, d_val = MLPData.regr1(1000, 0.0)

# Here we need to normalize the target values
norm_m = d_train.mean(axis=0)
norm_s = d_train.std(axis=0)
d_train = (d_train - norm_m) / norm_s

# We use the same normalization for the validation data.
d_val = (d_val - norm_m) / norm_s

In [58]:
config = {
    'model_params':{
        'inp_dim':x_train.shape[1],         
        'hidden_nodes':1,   # activation functions for the hidden layer
        'num_out':1 # if binary --> 1 |  regression--> num inputs | multi-class--> num of classes
    },
    'optimizer':'Adam',               # minimization method
    'criterion':'BCELoss', # error function
    'max_epochs':10,
}

In [60]:
%%time
# Get the model
model = Agent(MLP,**config)

# Train the model
trainer = pl.Trainer(
            max_epochs=config['max_epochs'], 
            gpus=-1 if torch.cuda.is_available() else None, 
            logger=metrics.MetricsLogger(),
            progress_bar_refresh_rate=1,
            weights_summary=None, # Can be None, top or full
            num_sanity_val_steps=10, 
            callbacks=[progressbar.LitProgressBar()]
        )

# Train the model
trainer.fit(
    model, 
    train_dataloader=train_loader,
    val_dataloaders=val_loader
);

# Call the stats function to print out statistics for classification problems
#pred_trn = model.predict(x_train).reshape(d_train.shape)
#pred_val = model.predict(x_val).reshape(d_val.shape)
plot.stats_reg(d_train, pred_trn, 'Training', estimator)
plot.stats_reg(d_val, pred_val, 'Validation', estimator)

# Scatter plots of predicted and true values
plt.figure()
plt.plot(d_train, pred_trn, 'g*', label='Predict vs True (Training)')
plt.legend()
plt.figure()
plt.plot(d_val, pred_val, 'b*', label='Predict vs True (Validation)')
plt.legend()
plt.show()

# Plot the learning curves
plt.figure()
plt.plot(estimator.history['loss'], label='Training')
plt.plot(estimator.history['val_loss'], label='Validation')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend()
plt.show()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


CPU times: user 1e+03 ns, sys: 3 µs, total: 4 µs
Wall time: 6.91 µs


NameError: name 'pred_trn' is not defined